In [46]:
from keras import layers, Input, Model
import glob,os
# from model import 

In [47]:
path_input_train = "../data/"
path_label_train = "../data/"
train_input = glob.glob(os.path.join(path_input_train,"*"))

In [48]:
def preprocess_image(image):
    """
    reads the image in RGB mode and  resizes it into 224,224. Also normalized the image pixel in range (0-1)
    :param image:
    :return: image
    """
    img = Image.open(image).convert('RGB')
    img_crop = img.resize((224,224), Image.ANTIALIAS)
    img_crop = np.array(img_crop)
    scaled_img = np.divide(img_crop,255.0)
    return scaled_img


def create_train_test_set():
    """
    create the data-set. Stores the input image and its corresponding labelled images in two numpy arrays as input and
    label. Also splits the whole data-set into train and test.
    :return: train_input, train_label, test_input, test_label
    """
    train_input_images, train_label_images = [], []

    for image in train_input:
        image_name = image.split("/")[-1]
        if os.path.exists(os.path.join(path_label_train,image_name)):
            train_input_images.append(preprocess_image(image))
            train_label_images.append(preprocess_image(os.path.join(path_label_train,image_name)))

    train_input_images = np.array(train_input_images)
    train_label_images = np.array(train_label_images)
    #test_input_images = train_input_images[-100:]
    #test_label_images = train_label_images[-100:]
    #train_input_images = train_input_images[:-100]
    #train_label_images = train_label_images[:-100]

    print("train_images_length",len(train_input_images),len(train_label_images))
#     print("train_images_length",len(train_input_images))
    #print("test_images_length",len(test_input_images),len(test_label_images))
    #return train_input_images, train_label_images, test_input_images, test_label_images
    return train_input_images, train_label_images



In [50]:
inputTensor = Input((224,224,1))

conv1 = layers.Conv2D(96,(11,11),strides=1,activation='relu')(inputTensor)

conv1_norm = layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001,center=True,
            scale=True, beta_initializer='zeros', gamma_initializer='ones',
            moving_mean_initializer='zeros',moving_variance_initializer='ones')(conv1)

conv1_pool = layers.MaxPool2D((3,3), 2)(conv1_norm)

conv2_padding = layers.ZeroPadding2D((2, 2))(conv1_pool)

conv2 = layers.Conv2D(256,(5,5),strides=1,activation='relu')(conv2_padding)
conv2_norm = layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001,center=True,
            scale=True, beta_initializer='zeros', gamma_initializer='ones',
            moving_mean_initializer='zeros',moving_variance_initializer='ones')(conv2)

conv2_pool = layers.MaxPool2D((3,3), 2)(conv2_norm)

conv2_dropout = layers.Dropout(0.3, seed=1)(conv2_pool)

conv3_padding = layers.ZeroPadding2D((1,1))(conv2_dropout)

conv3 = layers.Conv2D(384, (3,3), strides = 1, activation = 'relu')(conv3_padding)

conv4_padding = layers.ZeroPadding2D((1,1))(conv3)

conv4 = layers.Conv2D(256, (3,3), strides = 1, activation = 'relu')(conv4_padding)
                                                                    
conv4_pool = layers.MaxPool2D((3,3), 2)(conv4)
                                                                    
conv4_dropout = layers.Dropout(0.3, seed=1)(conv4_pool)
                                                                    
flatten_layer = layers.Flatten()(conv4_dropout)
                                                                    
fully_connected1 = layers.Dense(1024)(flatten_layer)

fc1_dropout = layers.Dropout(0.5, seed=1)(fully_connected1)
                                                                    
fully_connected2 = layers.Dense(128)(fc1_dropout)
                                                                    
similarity_model = Model(inputs = [inputTensor], outputs = fully_connected2, name = 'Similarity_Model')
                                                                    
similarity_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_15 (InputLayer)        (None, 224, 224, 1)       0         
_________________________________________________________________
conv2d_39 (Conv2D)           (None, 214, 214, 96)      11712     
_________________________________________________________________
batch_normalization_20 (Batc (None, 214, 214, 96)      384       
_________________________________________________________________
max_pooling2d_27 (MaxPooling (None, 106, 106, 96)      0         
_________________________________________________________________
zero_padding2d_25 (ZeroPaddi (None, 110, 110, 96)      0         
_________________________________________________________________
conv2d_40 (Conv2D)           (None, 106, 106, 256)     614656    
_________________________________________________________________
batch_normalization_21 (Batc (None, 106, 106, 256)     1024      
__________

In [69]:
in_dim=(224,224,1)
input_anchor = Input(shape=(in_dim))
input_positive = Input(shape=(in_dim))
input_negative = Input(shape=(in_dim))
processed_a=similarity_model(input_a)
processed_p=similarity_model(input_p)
processed_n=similarity_model(input_n)

In [70]:
processed_a.get_shape

<bound method Tensor.get_shape of <tf.Tensor 'Similarity_Model_3/dense_14/BiasAdd:0' shape=(?, 128) dtype=float32>>

In [71]:
import keras.backend as K


def euclidean_distance_loss(y_true, y_pred):
    """
    :param y_true: TensorFlow/Theano tensor
    :param y_pred: TensorFlow/Theano tensor of the same shape as y_true
    :return: float
    """
    return K.sqrt(K.sum(K.square(y_pred - y_true), axis=-1))

In [72]:
positive_dist= euclidean_distance_loss(processed_a, processed_p)
negative_dist = euclidean_distance_loss(processed_a, processed_n)


In [73]:
stacked_dists = layers.Lambda(lambda vects: K.stack(vects, axis=1),name='stacked_dists')
([positive_dist, negative_dist])

model = Model([input_anchor, input_positive, input_negative], stacked_dists, name='triple_siamese')


ValueError: Output tensors to a Model must be the output of a Keras `Layer` (thus holding past layer metadata). Found: <keras.layers.core.Lambda object at 0x7feca80469d0>